# Classical Simulation of Quantum Circuits

Performing a quantum computation (commonly described as a quantum circuit) entails evolving an initial quantum state by applying a sequence of operations (also called gates) and measuring the resulting system.
Eventually, the goal should obviously be to do that on a real device.
However, there are several important reasons for simulating the corresponding computations on a classical machine, particularly in the early stages of the design:
As long as no suitable devices are available (e.g., in terms of scale, feasible computation depth, or accuracy), classical simulations of quantum circuits still allow one to explore and test quantum applications, even if only on a limited scale.
However, also with further progress in the capabilities of the hardware platforms, classical simulation will remain an essential part of the quantum computing design process, since it additionally allows access to _all_ amplitudes of a resulting quantum state in contrast to a real device that only probabilistically returns measurement results.
Moreover, classical simulation provides means to study quantum error correction as well as a baseline to estimate the advantage of quantum computers over classical computers.

The classical simulation of quantum circuits is commonly conducted by performing consecutive matrix-vector multiplication, which many simulators realize by storing a dense representation of the complete state vector in memory and evolving it correspondingly or by relying on tensor network methods.
This approach quickly becomes intractable due to the exponential growth of the quantum state with respect to the number of qubits---quickly rendering such simulations infeasible even on supercomputer clusters.
Simulation methodologies based on decision diagrams are a promising complementary approach that frequently allows reducing the required memory by exploiting redundancies in the simulated quantum state.

The _MQT_ offers the classical quantum circuit simulator _DDSIM_ that can be used to perform various quantum circuit simulation tasks based on using decision diagrams as a data structure.
This includes strong and weak simulation, approximation techniques, noise-aware simulation, hybrid Schrödinger-Feynman techniques, support for dynamic circuits, the computation of expectation values, the simulation of mixed-dimensional systems, and more.

In the following, we will use IBM's Qiskit to define quantum circuits and then simulate them using the MQT DDSIM simulator and IBM's Aer simulator.

## Simulating Simple Quantum Gates

In [ ]:
from qiskit import QuantumCircuit

circ = QuantumCircuit(1)
circ.x(0)
circ.draw(output="mpl")

In [ ]:
from mqt.ddsim import DDSIMProvider
from qiskit.visualization import plot_histogram

provider = DDSIMProvider()
backend = provider.get_backend("qasm_simulator")
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Simulating a CNOT gate

In [ ]:
circ = QuantumCircuit(2)
circ.x(0)
circ.cx(0,1)
circ.measure_all()
circ.draw(output="mpl")

In [ ]:
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Simulating a Hadamard Gate and a Bell State

In [ ]:
circ = QuantumCircuit(2)
circ.h(0)
circ.draw(output="mpl")

In [ ]:
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

In [ ]:
circ.cx(0,1)
circ.measure_all()
circ.draw(output="mpl")

In [ ]:
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Simulating a GHZ State

In [ ]:
circ = QuantumCircuit(3)
circ.h(2)
circ.cx(2, 1)
circ.cx(1, 0)
circ.measure_all()
circ.draw(output="mpl")

In [ ]:
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

# MQT Bench

In [ ]:
from mqt.bench import get_benchmark
circ = get_benchmark(benchmark_name="ghz", level="alg", circuit_size=3)
circ.draw(output="mpl")

## Select a 5 qubit Quantum Phase Estimation (QPE) circuit and simulate it using DDSIM.

MQT Bench GitHub: https://github.com/cda-tum/mqt-bench

Documentation: https://mqt.readthedocs.io/projects/bench/en/latest/Benchmark_selection.html

In [ ]:
from mqt.bench import get_benchmark
circ = get_benchmark(benchmark_name="qpeinexact", level="alg", circuit_size=5)
circ.draw(output="mpl")

result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Evaluating the Runtime for a regular Simulator

In [ ]:
from qiskit_aer import AerProvider, AerSimulator
backend = AerSimulator(method="statevector")
runtimes = []
value_range = range(10, 30, 2)
for i in value_range:
    circ = get_benchmark(benchmark_name="ghz", level="alg", circuit_size=i)
    result = backend.run(circ, shots=10000).result()
    runtimes.append(result.time_taken)
    print(f"GHZ-{i}: {result.time_taken}")

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel("Circuit Size")
plt.ylabel("Runtime (s)")
plt.plot(value_range, runtimes)

## Evaluating the Runtime for the DDSim Simulator

In [ ]:
backend = provider.get_backend("qasm_simulator")
runtimes_ddsim = []
for i in value_range:
    circ = get_benchmark(benchmark_name="ghz", level="alg", circuit_size=i)
    result = backend.run(circ, shots=10000).result()
    runtimes_ddsim.append(result.time_taken)
    print(f"GHZ-{i}: {result.time_taken}")
plt.xlabel("Circuit Size")
plt.ylabel("Runtime (s)")
plt.plot(runtimes_ddsim)

## Direct Comparison

In [ ]:
plt.xlabel("Circuit Size")
plt.ylabel("Runtime (s)")
plt.title("Simulation Runtimes")
plt.plot(value_range, runtimes, label="Aer")
plt.plot(value_range, runtimes_ddsim, label="DDSim")
plt.legend()

## Inspiration for further tasks:

- Choose different algorithms from MQT Bench
- Use other Qiskit Aer Simulation Methods (see https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html)
- Use other MQT DDSim simulation methods (see https://mqt.readthedocs.io/projects/ddsim/en/latest/Simulators.html)